# TV Script Generation
In this project, I'll generate my own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  I'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network I'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
I'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
We can play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  We need to implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, I first need to transform the words to ids.  In this function, I create two dictionaries:
- A Dictionary to go from the words to an id, we'll call `vocab_to_int`
- A Dictionary to go from the id to word, we'll call `int_to_vocab`

I will return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Creating a lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Implementing Function
    vocab = sorted(set(text))
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
I'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

I will implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  I will create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. We need to make sure we don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [54]:
def token_lookup():
    """
    Generating a dict to turn punctuation into a token.
    return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # Implementing Function
    
    Tokens = { '.'  : '||Period||',
               ','  : '||Comma||',
               '"'  : 'QuotationMark',
               ';'  : 'Semicolon',
               '!'  : 'Exclamationmark',
               '?'  : 'Questionmark',
               '('  : 'LeftParentheses',
               ')'  : 'RightParentheses',
               '--' : 'Dash',
               '\n' : 'Return'
             }
    
    return Tokens  

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
WE DON'T NEED TO MODIFY ANYTHING IN THIS CELL
"""
# Preprocessing Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is my first checkpoint. If ze ever decide to come back to this notebook or have to restart the notebook, we can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
WE DON'T NEED TO MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
I'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
WE DON'T NEED TO MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Checking TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Checking for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
I Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

I will Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # Implementing Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
I will Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
- Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

I will Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [4]:
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

    return drop


def get_init_cell(batch_size, rnn_size):
    """
    Creating an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # Implementing Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    num_layers = 1
    cell = tf.contrib.rnn.MultiRNNCell([lstm]*num_layers, tf.float32)
    #cell = tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)],  state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(initial_state, name='initial_state')
    
    return cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Applying embedding to `input_data` using TensorFlow.  I will Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # Implementing Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
I created a RNN Cell in the `get_init_cell()` function.  It is Time to use the cell to create a RNN.
- Building the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Applying the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

The function returns the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [55]:
def build_rnn(cell, inputs):
    """
    Creating a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # Implementing Function
    Outputs, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs, dtype=tf.float32)
    FinalState = tf.identity(final_state, name='final_state')
    
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Applying the functions I implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

The function returns the logits and final state in the following tuple (Logits, FinalState) 

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Building part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # Implementing Function
    embed_data = get_embed(input_data, vocab_size, embed_dim)
    outputs, FinalState = build_rnn(cell, embed_data)
    Logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn = None) #1=vocab_size lessons??
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implementing `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If we can't fill the last batch with enough data, we need to drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

We need to notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [57]:

def get_batches(int_text, batch_size, seq_length):
    """
    Returning batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # Implementing Function
    n_batches = (len(int_text) // (batch_size * seq_length)) 
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    ydata[-1] = xdata[0]
    
    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    
    # print (batches)
    
    return batches

get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

[[[[ 1  2]
   [ 7  8]
   [13 14]]

  [[ 2  3]
   [ 8  9]
   [14 15]]]


 [[[ 3  4]
   [ 9 10]
   [15 16]]

  [[ 4  5]
   [10 11]
   [16 17]]]


 [[[ 5  6]
   [11 12]
   [17 18]]

  [[ 6  7]
   [12 13]
   [18  1]]]]
[[[[   0    1    2    3    4]
   [  35   36   37   38   39]
   [  70   71   72   73   74]
   ...
   [4375 4376 4377 4378 4379]
   [4410 4411 4412 4413 4414]
   [4445 4446 4447 4448 4449]]

  [[   1    2    3    4    5]
   [  36   37   38   39   40]
   [  71   72   73   74   75]
   ...
   [4376 4377 4378 4379 4380]
   [4411 4412 4413 4414 4415]
   [4446 4447 4448 4449 4450]]]


 [[[   5    6    7    8    9]
   [  40   41   42   43   44]
   [  75   76   77   78   79]
   ...
   [4380 4381 4382 4383 4384]
   [4415 4416 4417 4418 4419]
   [4450 4451 4452 4453 4454]]

  [[   6    7    8    9   10]
   [  41   42   43   44   45]
   [  76   77   78   79   80]
   ...
   [4381 4382 4383 4384 4385]
   [4416 4417 4418 4419 4420]
   [4451 4452 4453 4454 4455]]]


 [[[  10   11   12   13  

## Neural Network Training
### Hyperparameters
Tuning the following parameters:

- Setting `num_epochs` to the number of epochs.
- Setting `batch_size` to the batch size.
- Setting `rnn_size` to the size of the RNNs.
- Setting `embed_dim` to the size of the embedding.
- Setting `seq_length` to the length of sequence.
- Setting `learning_rate` to the learning rate.
- Setting `show_every_n_batches` to the number of batches the neural network should print progress.

In [44]:
# Number of Epochs
num_epochs = 250
# Batch Size
batch_size = 320
# RNN Size
rnn_size = 400
# Embedding Dimension Size
embed_dim = 100
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Building the Graph
I Build the graph using the neural network you implemented.

In [45]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Training the neural network on the preprocessed data. 

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Showing every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.822
Epoch   0 Batch   10/21   train_loss = 7.809
Epoch   0 Batch   20/21   train_loss = 6.857
Epoch   1 Batch    9/21   train_loss = 6.603
Epoch   1 Batch   19/21   train_loss = 6.390
Epoch   2 Batch    8/21   train_loss = 6.270
Epoch   2 Batch   18/21   train_loss = 6.109
Epoch   3 Batch    7/21   train_loss = 5.956
Epoch   3 Batch   17/21   train_loss = 5.889
Epoch   4 Batch    6/21   train_loss = 5.724
Epoch   4 Batch   16/21   train_loss = 5.648
Epoch   5 Batch    5/21   train_loss = 5.574
Epoch   5 Batch   15/21   train_loss = 5.603
Epoch   6 Batch    4/21   train_loss = 5.596
Epoch   6 Batch   14/21   train_loss = 5.450
Epoch   7 Batch    3/21   train_loss = 5.414
Epoch   7 Batch   13/21   train_loss = 5.457
Epoch   8 Batch    2/21   train_loss = 5.182
Epoch   8 Batch   12/21   train_loss = 5.330
Epoch   9 Batch    1/21   train_loss = 5.284
Epoch   9 Batch   11/21   train_loss = 5.213
Epoch  10 Batch    0/21   train_loss = 5.151
Epoch  10 

## Saving Parameters
Saving `seq_length` and `save_dir` for generating a new TV script.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Saving parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Getting Tensors
Getting tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Getting the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

The function returns the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [49]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
        
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choosing Word
Implementing the `pick_word()` function to select the next word using `probabilities`.

In [50]:
def pick_word(probabilities, int_to_vocab):
    """
    Picking the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # Implementing Function
    return np.random.choice(list(int_to_vocab.values()),p = probabilities)
    #https://discussions.udacity.com/t/big-picture-question-quality-of-the-final-script/235778/4 for explanation


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generating TV Script
This will generate the TV script.  Setting `gen_length` to the length of TV script you want to generate.

In [51]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Loading saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Getting Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generating sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Getting Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Removing tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(warily) i dunno if i want you in the sequel.(sighs)
homer_simpson:(pained) big deal. are you really think of my money?
homer_simpson: well, i'm not a good...


moe_szyslak:(spinning) it tastes like one of those weird.(laughs)

grampa_simpson: i keep us down so then.
homer_simpson:(super casual) quiet, but i'm not gonna gamble... and we light them what more of for the big deal.
barney_gumble: i'm a thing who's generally like
barney_gumble:(lying) she took out of vodka.


moe_szyslak: see? uh-oh, the bees, homer?
bart_simpson: no, don't be sure.
homer_simpson: listen, moe, a little girl water. you think it was better than me.
stillwater: y'know, i just spend it all.
homer_simpson: mmmm, danish.
moe_szyslak: now, let's get out.
homer_simpson: oh, i'm not your soul. i saw her super bowl.(to bully


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, I'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have to train on all the data, because that would take too long.  However, ze are free to train our neural network on all the data.